In [138]:
import numpy as np
import pandas as pd

In [139]:
song_df = pd.read_csv("Spotify_song.csv", encoding="ISO-8859-1")
song_df.head(3)

,artist,song,link
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html


In [140]:
song_df.shape

(57650, 3)

In [141]:
song_df = song_df.sample(1700).drop("link", axis = 1).reset_index(drop = True)

In [142]:
song_df.isnull().sum()

artist    0
song      0
dtype: int64

In [143]:
song_df.shape

(1700, 2)

In [144]:
song_df["song"] = song_df["song"].str.lower()
song_df["song"] = song_df["song"].replace(r"^\w\s", " ")
song_df["song"] = song_df["song"].replace(r"\n", " ", regex = True)

In [145]:
song_df.head(3)

,artist,song
0,Les Miserables,the runaway cart
1,Kiss,move on
2,Journey,one more


In [146]:
import nltk
nltk.download("punkt_tab")
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt_tab to C:\Users\ANKIT
[nltk_data]     THAKUR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [147]:
ps = PorterStemmer()

In [148]:
def token(txt):
    token = nltk.word_tokenize(txt)
    wt = [ps.stem(w) for w in token]
    return " ".join(wt)

In [149]:
song_df["song"].apply(lambda x: token(x))

0                    the runaway cart
1                             move on
2                            one more
3               what i like about you
4       children go where i send thee
                    ...              
1695                       cherri pie
1696        rock 'n ' roll pain train
1697                      dirti women
1698                     classic rock
1699                 intim get togeth
Name: song, Length: 1700, dtype: object

In [150]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [151]:
cv = CountVectorizer(analyzer = "word", stop_words = "english")
matrix = cv.fit_transform(song_df["song"])

In [152]:
cs_similar = cosine_similarity(matrix)

In [153]:
def recommender(artist_name):
    artist_songs = song_df[song_df["artist"] == artist_name]
    if artist_songs.empty:
        return f"No songs found"
    recommendations = artist_songs["song"].head(5).tolist()
    return recommendations


In [154]:
recommender("Taylor Swift")

['somebody to hold', 'mine', 'breathless', 'silent night']

In [155]:
import pickle
pickle.dump(cs_similar, open("Song_Recommedation.pkl", "wb"))

In [156]:
dataset_sample = song_df[['artist', 'song']]
dataset_sample.to_csv("dataset_sample.csv", index=False, encoding="ISO-8859-1")